# Lesson 15: Multi-agent E-commerce Support System

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/harshit-vibes/lyzr-adk-demo/blob/master/notebooks/15_multi_agent_ecommerce.ipynb)


🔴 **Advanced** · ⏱ **35 min**

---

Real customer support pipelines rarely use a single agent. Complex queries may span orders,
billing, and product knowledge — domains that benefit from specialization.

In this lesson you will build a **manager + 3 specialist sub-agents** system for e-commerce
customer support. The manager receives every customer message and routes it to the right
specialist by calling it as a Python tool.

## What you'll learn

- Build multiple specialist sub-agents with distinct roles
- Wrap sub-agents as Python tool functions
- Add sub-agent tools to a manager agent
- Understand how the manager uses docstrings to route queries
- Handle multi-domain queries where the manager calls more than one sub-agent
- See the common mistake: vague docstrings cause incorrect routing

## Prerequisites

Before running this notebook, you should be familiar with:

- **Lesson 1** — Studio & `agent.run()` basics
- **Lesson 3** — Agent lifecycle (create, run, manage)
- **Lesson 6** — Custom tools — **especially docstrings** (critical for routing)
- **Lesson 5** — Memory & sessions (optional but helpful)

You also need your **`LYZR_API_KEY`** set as an environment variable:

```bash
export LYZR_API_KEY="your-api-key-here"
```

In [ ]:
!pip install lyzr-adk -q

In [ ]:
import os
from lyzr import Studio

API_KEY = os.getenv("LYZR_API_KEY", "YOUR_LYZR_API_KEY")
studio = Studio(api_key=API_KEY)
print("Ready!")

## 1. Architecture Overview

The **agent-as-tool** pattern is the simplest and most portable way to compose multiple agents.
Each specialist sub-agent is wrapped in a Python function and attached to the manager agent
as a regular tool.

```
Customer query
       │
       ▼
  Manager Agent  ──reads docstrings──▶  picks tool to call
       │
       ├──▶ handle_order_query()   ──▶  OrderAgent   (orders, returns, shipping)
       ├──▶ handle_product_query() ──▶  ProductAgent  (specs, availability, recommendations)
       └──▶ handle_billing_query() ──▶  BillingAgent  (payments, refunds, invoices)
```

**Why this pattern works:**
- Each sub-agent has a focused role → better, more accurate answers
- The manager never needs to know the implementation details of each specialist
- The docstring on each wrapper function is what guides the manager's routing decision
- Adding a new specialist = create agent + write wrapper function + `manager.add_tool(fn)` — nothing else changes

## 2. Creating the Specialist Sub-agents

Create three sub-agents, each with a tightly focused role, goal, and instructions.
The narrower the focus, the more reliably each sub-agent answers questions in its domain.

In [ ]:
# Sub-agent 1: Order specialist
order_agent = studio.create_agent(
    name="Order Specialist",
    provider="openai/gpt-4o",
    role="E-commerce order management specialist",
    goal="Resolve all customer questions about orders, returns, cancellations, and shipping",
    instructions=(
        "You specialize exclusively in order management. "
        "For order status queries, always mention the order ID in your response. "
        "For return requests, confirm the return window (30 days) and the process. "
        "For cancellations, check if the order has shipped; if so, initiate a return instead. "
        "Keep responses concise and action-oriented."
    )
)
print(f"OrderAgent: {order_agent.id}")

# Sub-agent 2: Product specialist
product_agent = studio.create_agent(
    name="Product Specialist",
    provider="openai/gpt-4o",
    role="E-commerce product and inventory specialist",
    goal="Answer all customer questions about products, specifications, availability, and recommendations",
    instructions=(
        "You specialize exclusively in product knowledge. "
        "When asked about availability, always give a concrete answer (in stock / out of stock / pre-order). "
        "For product comparisons, use a clear table format. "
        "For recommendations, ask one clarifying question if needed, then suggest 2-3 options with reasoning. "
        "Never speculate about pricing or shipping — route those to the appropriate team."
    )
)
print(f"ProductAgent: {product_agent.id}")

# Sub-agent 3: Billing specialist
billing_agent = studio.create_agent(
    name="Billing Specialist",
    provider="openai/gpt-4o",
    role="E-commerce billing and payments specialist",
    goal="Resolve all billing issues, process refund requests, and answer payment and invoice questions",
    instructions=(
        "You specialize exclusively in billing and payments. "
        "For refund requests, confirm the amount and timeline (5-7 business days to original payment method). "
        "For payment failures, check for common causes (expired card, insufficient funds, bank block). "
        "For invoice requests, confirm what information is needed and the format available. "
        "Never discuss order logistics or product details — those are handled by other teams."
    )
)
print(f"BillingAgent: {billing_agent.id}")

## 3. Wrapping Sub-agents as Tools

Each sub-agent is wrapped in a Python function. This function:

1. Calls `sub_agent.run(query)` and returns the response string
2. Has a **precise docstring** — this is what the manager LLM reads to decide routing
3. Has a single `query: str` parameter — the manager extracts and passes the user's question

**The docstring is the routing logic.** Make it specific about what types of questions belong to each specialist. Vague or overlapping docstrings cause the manager to pick the wrong sub-agent.

In [ ]:
def handle_order_query(query: str) -> str:
    """Handle customer questions about orders, returns, cancellations, and shipping status.

    Use this tool for:
    - Order status and tracking ("Where is my order?", "Has ORD-1001 shipped?")
    - Return requests ("I want to return my purchase")
    - Order cancellations ("Can I cancel order ORD-1002?")
    - Delivery timeframe questions ("When will it arrive?")

    Do NOT use for product questions, billing issues, or payment problems.
    """
    response = order_agent.run(query)
    return response.response


def handle_product_query(query: str) -> str:
    """Answer customer questions about products, availability, specifications, and recommendations.

    Use this tool for:
    - Product availability ("Is the blue backpack in stock?")
    - Product specifications ("What are the dimensions of the standing desk?")
    - Product comparisons ("What's the difference between Model A and Model B?")
    - Purchase recommendations ("I need a laptop for video editing under $1500")

    Do NOT use for order status, billing, or payment questions.
    """
    response = product_agent.run(query)
    return response.response


def handle_billing_query(query: str) -> str:
    """Resolve billing issues, process refund requests, and answer payment and invoice questions.

    Use this tool for:
    - Refund requests ("I'd like a refund for my order")
    - Payment failures ("My card was declined")
    - Invoice requests ("I need an invoice for my purchase")
    - Billing statement questions ("I was charged twice")

    Do NOT use for order logistics, shipping, or product information.
    """
    response = billing_agent.run(query)
    return response.response


print("Wrapper functions defined: handle_order_query, handle_product_query, handle_billing_query")

## 4. Creating the Manager Agent

The manager agent:
- Has **no knowledge base, no RAG, no specialized knowledge** of its own
- Has all three wrapper functions added as tools
- Has instructions to route carefully based on query intent
- Will call multiple tools if a query spans domains

The manager's `instructions` should emphasize routing precision. It should never try to answer domain questions itself — its job is to understand intent and delegate.

In [ ]:
manager_agent = studio.create_agent(
    name="Support Manager",
    provider="openai/gpt-4o",
    role="Customer support routing manager",
    goal="Route customer queries to the correct specialist and synthesize their responses",
    instructions=(
        "You are a support routing manager. You do NOT answer questions yourself. "
        "Always use the appropriate specialist tool for every customer query. "
        "For questions about orders or shipping: use handle_order_query. "
        "For questions about products or availability: use handle_product_query. "
        "For billing, payments, or refunds: use handle_billing_query. "
        "If a query spans multiple domains (e.g., order AND refund), call both relevant tools "
        "and synthesize their answers into a single coherent response. "
        "Never guess or answer from your own knowledge — always use a tool."
    )
)

manager_agent.add_tool(handle_order_query)
manager_agent.add_tool(handle_product_query)
manager_agent.add_tool(handle_billing_query)

print(f"Manager agent ready: {manager_agent.id}")
print("Tools: handle_order_query, handle_product_query, handle_billing_query")

## 5. End-to-end Examples

Let's run four scenarios through the manager:
1. A pure order query (routes to OrderAgent)
2. A pure product query (routes to ProductAgent)
3. A pure billing query (routes to BillingAgent)
4. An ambiguous multi-domain query (manager calls two tools and synthesizes)

In [ ]:
# Example 1: Order query → routes to OrderAgent
print("=" * 60)
print("QUERY 1: Order status")
r1 = manager_agent.run("Hi, I placed order ORD-1001 last week. Can you tell me where it is?")
print(r1.response)

# Example 2: Product query → routes to ProductAgent
print("\n" + "=" * 60)
print("QUERY 2: Product recommendation")
r2 = manager_agent.run("I'm looking for a wireless keyboard for programming. Budget is $150. Any recommendations?")
print(r2.response)

# Example 3: Billing query → routes to BillingAgent
print("\n" + "=" * 60)
print("QUERY 3: Refund request")
r3 = manager_agent.run("I was charged twice for my last order. I'd like a refund for the duplicate charge.")
print(r3.response)

# Example 4: Multi-domain → manager calls both OrderAgent AND BillingAgent
print("\n" + "=" * 60)
print("QUERY 4: Multi-domain (order + refund)")
r4 = manager_agent.run("My order ORD-1002 still hasn't arrived and I want to cancel it and get a refund.")
print(r4.response)

## Common Mistake: Vague Tool Docstrings

If the docstrings on the wrapper functions are vague or overlapping, the manager picks the
wrong specialist — and you get incorrect or irrelevant answers.

**Bad docstring (vague):**
```python
def handle_order_query(query: str) -> str:
    """Handle customer questions."""  # ❌ too vague — what questions?
```

**Better docstring (specific):**
```python
def handle_order_query(query: str) -> str:
    """Handle questions about orders, returns, cancellations, and shipping status.
    Use for: order tracking, return requests, delivery questions.
    Do NOT use for: product specs, payment issues."""  # ✅ clear boundaries
```

**Rule:** Each tool's docstring must:
1. State clearly what kinds of queries it handles
2. Give 2-4 concrete examples
3. Say what it does NOT handle (avoids overlap with other tools)

In [ ]:
# ❌ Mistake: vague docstrings — the manager cannot distinguish tools

def bad_order_tool(query: str) -> str:
    """Handle customer questions."""  # ❌ too vague — could be anything
    result = order_agent.run(query).response
    print("  [bad_order_tool was called]")   # ← visible trace
    return result

def bad_product_tool(query: str) -> str:
    """Help customers."""  # ❌ too vague — could be anything
    result = product_agent.run(query).response
    print("  [bad_product_tool was called]")  # ← visible trace
    return result

bad_manager = studio.create_agent(
    name="Bad Manager Demo",
    provider="openai/gpt-4o",
    role="Support manager",
    goal="Route customer queries",
    instructions="Use the appropriate tool for each query."
)
bad_manager.add_tool(bad_order_tool)
bad_manager.add_tool(bad_product_tool)

# Run two queries that need different tools
# With precise docstrings the routing would be correct.
# With vague docstrings the manager can't tell the difference.

print("Query 1: order question (should call bad_order_tool)")
r1 = bad_manager.run("Where is my order ORD-1001?")
print(f"Response: {r1.response[:100]}...\n")

print("Query 2: product question (should call bad_product_tool)")
r2 = bad_manager.run("What wireless keyboards do you carry?")
print(f"Response: {r2.response[:100]}...\n")

print("✅ Fix: Write specific docstrings with 'Use this tool for' and 'Do NOT use for' sections.")
print("   With vague docstrings above, the manager may call the same tool for both queries.")
print("   Check the [tool was called] traces above to see what actually happened.")

## Exercise: Add a Shipping Agent

Your task is to extend the system with a fourth specialist — a shipping agent that
handles shipping cost estimates and delivery time predictions.

**Steps:**

1. Create a `ShippingAgent` with an appropriate role, goal, and instructions
2. Write a `handle_shipping_query(query: str) -> str` wrapper with a precise docstring
3. Add it to `manager_agent` with `manager_agent.add_tool(handle_shipping_query)`
4. Test with: "How much does express shipping to New York cost?" and verify the manager routes to your new agent

Fill in the `...` placeholders below.

In [ ]:
# TODO: Create the shipping specialist sub-agent
shipping_agent = studio.create_agent(
    name=...,
    provider="openai/gpt-4o",
    role=...,
    goal=...,
    instructions=...
)

# TODO: Write a wrapper function with a precise docstring
def handle_shipping_query(query: str) -> str:
    """..."""  # TODO: Be specific about what shipping questions this handles
    # TODO: Call shipping_agent.run(query) and return the response string
    ...


# TODO: Add the new tool to the manager
# manager_agent.add_tool(handle_shipping_query)

# TODO: Test the routing
# r = manager_agent.run("How much does express shipping to New York cost?")
# print(r.response)

## Summary

### Multi-agent Architecture Patterns

| Pattern | When to use | Complexity |
|---|---|---|
| **Agent-as-tool** (this lesson) | Each sub-agent is independent, manager routes | Low — uses existing `add_tool()` |
| Parallel agents | Sub-agents run concurrently, results merged | Medium |
| Pipeline agents | Each agent passes output to next | Medium |
| Hierarchical (nested managers) | Large systems with many domains | High |

### Key Takeaways

- **Agent-as-tool** = wrap `sub_agent.run(query)` in a Python function, attach to manager with `add_tool()`
- The **docstring on each wrapper** is the routing logic — make it precise with examples and exclusions
- The manager needs `instructions` that enforce tool-first routing (never answer from knowledge)
- For multi-domain queries, the manager automatically calls multiple tools and synthesizes the result
- Adding a new specialist: create agent + write wrapper + `manager.add_tool(fn)` — nothing else changes

### Quick Reference

```python
# 1. Create sub-agents
specialist = studio.create_agent(name="...", provider="openai/gpt-4o", role="...", ...)

# 2. Wrap as tool
def handle_specialist_query(query: str) -> str:
    """Specific docstring describing what queries this handles."""
    return specialist.run(query).response

# 3. Add to manager
manager.add_tool(handle_specialist_query)

# 4. Run — manager routes automatically
response = manager.run("customer message")
```

## Next Steps

You have completed all 15 lessons in the lyzr-adk series.

**What to build next:**
- Combine Lesson 14 + Lesson 15: a multi-agent WhatsApp bot where the manager routes messages to order, product, and billing sub-agents
- Add RAG (Lesson 7) to the ProductAgent so it answers from your real product catalog
- Add memory (Lesson 5) to the manager so customers can refer back to earlier questions

---

| Lesson | Topic |
|---|---|
| [01](./01_getting_started.ipynb) | Getting Started |
| [06](./06_tools_and_functions.ipynb) | Tools & Functions |
| [07](./07_knowledge_bases_rag.ipynb) | Knowledge Bases (RAG) |
| [10](./10_capstone_project.ipynb) | Capstone Project |
| [14](./14_whatsapp_business_copilot.ipynb) | WhatsApp Co-pilot |
| **15** | **Multi-agent E-commerce (this lesson)** |